### Import necessary libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.utils import np_utils
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from PIL import Image, ImageOps

### Load the data

In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

11501568/11490434 [==============================] - 1s 0us/step


### Data pre-processing

In [3]:
X_train = X_train.reshape(60000, 28, 28, 1).astype('float32')
X_test = X_test.reshape(10000, 28, 28, 1).astype('float32')

In [4]:
number_of_classes = 10
Y_train = np_utils.to_categorical(y_train, number_of_classes)
Y_test = np_utils.to_categorical(y_test, number_of_classes)

### Create model

In [5]:
model = Sequential()
model.add(Conv2D(64, (3, 3), input_shape=(28, 28, 1), activation="relu"))
model.add(Conv2D(32, (3, 3), activation="relu"))
model.add(Flatten())
model.add(Dense(number_of_classes, activation="softmax"))
    
model.compile(loss='categorical_crossentropy', optimizer="Adam", metrics=["accuracy"])

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 64)        640       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        18464     
                                                                 
 flatten (Flatten)           (None, 18432)             0         
                                                                 
 dense (Dense)               (None, 10)                184330    
                                                                 
Total params: 203,434
Trainable params: 203,434
Non-trainable params: 0
_________________________________________________________________


### Train the model

In [7]:
model.fit(X_train, Y_train, batch_size=32, epochs=5, validation_data=(X_test,Y_test))

Epoch 1/5
1875/1875 [==============================] - 162s 86ms/step - loss: 0.2082 - accuracy: 0.9516 - val_loss: 0.0860 - val_accuracy: 0.9760
Epoch 2/5
1875/1875 [==============================] - 161s 86ms/step - loss: 0.0727 - accuracy: 0.9776 - val_loss: 0.0894 - val_accuracy: 0.9734
Epoch 3/5
1875/1875 [==============================] - 160s 86ms/step - loss: 0.0514 - accuracy: 0.9834 - val_loss: 0.0920 - val_accuracy: 0.9742
Epoch 4/5
1875/1875 [==============================] - 161s 86ms/step - loss: 0.0383 - accuracy: 0.9880 - val_loss: 0.0976 - val_accuracy: 0.9774
Epoch 5/5
1875/1875 [==============================] - 161s 86ms/step - loss: 0.0317 - accuracy: 0.9897 - val_loss: 0.1177 - val_accuracy: 0.9743


In [8]:
metrics = model.evaluate(X_test, Y_test, verbose=0)
print("Metrics (Test Loss & Test Accuracy): ")
print(metrics)

Metrics (Test Loss & Test Accuracy): 
[0.11765547096729279, 0.9743000268936157]


### Save the model

In [9]:
model.save("model.h5")

In [10]:
!tar -zcvf model.tgz model.h5

model.h5


### Install necessary packages

In [11]:
!pip install watson-machine-learning-client

     |████████████████████████████████| 538 kB 13.2 MB/s eta 0:00:01


### Connect to IBM Watson Machine Learning instance

In [18]:
from ibm_watson_machine_learning import APIClient

API_KEY = "qyrJtdFzG7YfxKOEng8P0KKNr7-rLFOU7IvWtqfZKGQN"

credentials = {
    "url": "https://eu-gb.ml.cloud.ibm.com",
    "apikey": API_KEY
}

client = APIClient(credentials)

In [19]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']['name'] == space_name)['metadata']['id'])

In [20]:
space_uid = guid_from_space_name(client, 'Handwritten Digit Recognition')
print("Space UID: ", space_uid)

Space UID:  db99357d-f95e-4fa8-ba20-bfb784267cae


In [21]:
client.set.default_space(space_uid)

'SUCCESS'

### Define model specifications for deployment

In [22]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [23]:
software_spec_uid = client.software_specifications.get_uid_by_name("runtime-22.1-py3.9")
software_spec_uid

'12b83a17-24d8-5082-900f-0ab31fbfd3cb'

In [24]:
model_details = client.repository.store_model(model="model.tgz", meta_props={
    client.repository.ModelMetaNames.NAME: "CNN",
    client.repository.ModelMetaNames.TYPE: "tensorflow_2.7",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid
})

model_id = client.repository.get_model_id(model_details)

In [25]:
model_id

'ccd26c55-9d81-42aa-825c-44d265205407'

### Download the deployed model

In [26]:
client.repository.download(model_id, "model.tar.gz")

Successfully saved model content to file: 'model.tar.gz'


'/home/wsuser/work/model.tar.gz'